In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import dask
import os
import xskillscore as xs


In [2]:
#load in hfls data and also laod in lhf era5 data
e5_LH = xr.open_dataarray("/home/z5113258/Documents/data/REAN_HB/ERA5/ERA5_lhf.nc")
e5_SH = xr.open_dataarray("/home/z5113258/Documents/data/REAN_HB/ERA5/ERA5_shf.nc")
e5_SW = xr.open_dataarray("/home/z5113258/Documents/data/REAN_HB/ERA5/ERA5_swrf.nc")
e5_LW = xr.open_dataarray("/home/z5113258/Documents/data/REAN_HB/ERA5/ERA5_lwrf.nc")

            

In [4]:
def get_region(da, lat, lat_r, lon, lon_r):
    DA_ss = da.sel(lat=slice(lat, lat_r),lon=slice(lon, lon_r)).mean(['lat', 'lon'])
    return DA_ss

In [5]:
def save_rean_var(da, lat, lat1, lon, lon1, varname, file):
    DA = get_region(DA, lat, lat1, lon, lon1)
    DA = DA_.rename(var_name).to_dataframe()
    DA.to_csv("/home/z5113258/Documents/data/region_csv/"+file+".csv")
    return

In [ ]:
save_rean_var(e5_LH, -10, 10, 50, 70, "LH", "ERA5_LH")

In [3]:
outdir = '/home/z5113258/Documents/data/CMIP6_HB_new/'#dict with file names
models={}
varis = {'lw':'_netLW.nc', 'sw':'_netSW.nc', 'lhf':'_hfls.nc', 'shf':'_hfss.nc'}
for model in os.listdir(outdir):
    #go thorugh all varis
    if model in ['NESM3']:
        pass
    else:
        mod={}
        for key in varis.keys():
            #input file
            inp_files = (os.path.join(outdir, model, str(model+varis[key])))
            try:
                #load in file
                var = xr.open_dataarray(inp_files)
                #put into dict
                mod[key]=var
                models[model] = mod

            except Exception as e:
                print(e)
                pass



[Errno 2] No such file or directory: b'/home/z5113258/Documents/data/CMIP6_HB_new/not_full/not_full_netLW.nc'
[Errno 2] No such file or directory: b'/home/z5113258/Documents/data/CMIP6_HB_new/not_full/not_full_netSW.nc'
[Errno 2] No such file or directory: b'/home/z5113258/Documents/data/CMIP6_HB_new/not_full/not_full_hfls.nc'
[Errno 2] No such file or directory: b'/home/z5113258/Documents/data/CMIP6_HB_new/not_full/not_full_hfss.nc'


In [ ]:
def get_model_TS(MM_D):
    for i, model in enumerate(MM_D.keys()):
        HB = MM_D[model]
        T=HB['dTdt']
        U=HB['ua']
        V=HB['va']
        W=HB['wa']
        Q=HB['q']
        resid = HB['res']
        SST = HB['SST']

        T=T.assign_coords({'model':model})
        U=U.assign_coords({'model':model})
        V=V.assign_coords({'model':model})
        W=W.assign_coords({'model':model})
        Q=Q.assign_coords({'model':model})
        resid = resid.assign_coords({'model':model})
        SST = SST.assign_coords({'model':model})
        #now turn all these variables into an array
        if i == 0:
            TM = T
            UM = U
            VM = V
            WM = W
            QM = Q
            RM = resid
            SM = SST
        else:
            TM = xr.concat([TM, T], dim = 'model')
            UM = xr.concat([UM, U], dim = 'model')
            VM = xr.concat([VM, V], dim = 'model')
            WM = xr.concat([WM, W], dim = 'model')
            QM = xr.concat([QM, Q], dim = 'model')
            RM = xr.concat([RM, resid], dim = 'model')
            SM = xr.concat([SM, SST], dim = 'model')
    return TM, UM, VM, WM, QM, RM, SM